In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
class CART_reg():
       
    def fit(self):
        self.__input_train()
        X = self.X
        y = self.y

        self.__CART_fit(X,y)
        pass

    def predict(self):
        y_tst, X_tst = self.__input_test()


        y_pred = self.y_pred
        bestvariable = self.bestvariable
        bestcrit = self.bestcrit

        x_cl_tst = X_tst.iloc[:,bestvariable]

        y_hat = np.zeros(len(y_tst))


        if self.is_numeric:
            y_hat[x_cl_tst >= bestcrit] = y_pred[0]
            y_hat[x_cl_tst < bestcrit] = y_pred[1]

        else:
            idx = self.__find_subset_index2(bestcrit,x_cl_tst)
            notidx = [x for x in range(len(y_tst)) if not(x in idx)]
            y_hat[idx] = y_pred[0] ; y_hat[notidx] = y_pred[1]

        result = pd.DataFrame()
        result['Actual Class'] = y_tst
        result['Predicted Class'] = y_hat

        return result
        


    def print_results(self):
        out_name = input("Enter the output file name to export [(ex) result.txt] : ")
        MAE, MAPE, RMSE = self.__goodness_of_fit()
        X = self.X

        f = open(out_name, 'w+')
        f.write('Tree Structure  \n')
        f.write('---------------------------------\n')        
        if self.is_numeric:
            f.write(f'\tNode 1 : n = {len(self.y)}, mean = {np.round(np.mean(self.y),3)} \n')
            f.write(f'\t\tNode 2 : {X.columns[self.bestvariable]} >= {self.bestcrit} , n = {len(self.y_classified[0])}, mean = {np.round(self.y_pred[0],3)} \n')
            f.write(f'\t\tNode 3 : {X.columns[self.bestvariable]} < {self.bestcrit} , n = {len(self.y_classified[1])}, mean = {np.round(self.y_pred[1],3)} \n')
        else:
            f.write(f'\tNode 1 : n = {len(self.y)}, mean = {np.mean(self.y)} \n')
            f.write(f'\t\tNode 2 : {X.columns[self.bestvariable]} = {self.bestcrit}, n = {len(self.y_classified[0])}, mean = {np.round(self.y_pred[0])} \n')
            f.write(f'\t\tNode 3 : {X.columns[self.bestvariable]} = {[x for x in np.sort(X.iloc[:,self.bestvariable_cart].unique()) if not(x in self.bestcrit_cart)]},  n = {len(self.y_classified[1])}, mean = {np.round(self.y_pred[1])} \n')


        f.write('\n\n')
        f.write('Prediction Performance (Test)\n')
        f.write('---------------------------------\n')
        f.write(f'\tMAE = {MAE} \n')
        f.write(f'\tMAPE = {MAPE} \n')
        f.write(f'\tRMSE = {RMSE} \n')
        f.close()

        r = open(out_name,'r')
        for line in r.readlines():
            print(line)
        r.close()

        pass  

    
    def __goodness_of_fit(self):
        result = self.predict()
        MAE = np.sum(np.abs(result.iloc[:,0] - result.iloc[:,1])) / len(result)
        MAPE =  np.sum(np.abs((result.iloc[:,0] - result.iloc[:,1]) / result.iloc[:,0])) / len(result)
        RMSE = np.sqrt(np.sum((result.iloc[:,0] - result.iloc[:,1])**2)/len(result))

        return np.round(MAE,3),np.round(MAPE,3),np.round(RMSE,3)


    
    def __input_train(self):
        # input data file name
        train_data=input("Enter the name of training data file [(ex) boston_tr.csv] : ") # data name
        self.train_data = train_data
        coding_fm=int(input("Select the data Seperator(1 = ' ' or 2 = ','): ")) # data separator of training data
        self.coding_fm = coding_fm
        separator_fm={coding_fm ==1 : " "}.get(True, ",")
        self.separator_fm = separator_fm

        # input header & assign response variables
        self.header=input("Does the data have column header? (y/n) : ")
        header = self.header
        if(self.header=="y") :
            trdata=pd.read_csv(train_data, sep=separator_fm) # loading data with header
            self.trdata = trdata
            res_pos = input(f'Enter the column name of response variable among {list(trdata.columns)} : ')
            self.res_pos = res_pos           
            self.y = trdata[res_pos]
            self.X = trdata.drop(self.res_pos, axis = 1)

        else : 
            self.trdata=pd.read_csv(train_data, sep=separator_fm, header=None) # loading data without header
            trdata = self.trdata
            self.res_pos = int(input(f'Enter the column position of the response variable : \n [from 1 to {trdata.shape[1]}] : '))
            res_pos = self.res_pos
            X_index = []
            for i in range(len(trdata.columns)): 
                if i == res_pos-1 : continue
                X_index.append(i)
            self.y = trdata.iloc[:,res_pos-1]
            self.X = trdata.iloc[:,X_index]
    
    
    def __input_test(self):
        test_data =input("Enter the name of test data file [(ex) boston_tst.csv] : ")
        coding_fm = self.coding_fm
        separator_fm = self.separator_fm
        header = self.header
        res_pos = self.res_pos
        
        if(self.header=="y") :
            tstdata=pd.read_csv(test_data, sep=separator_fm) # loading data with header
            y_tst = tstdata[res_pos]
            X_tst = tstdata.drop(self.res_pos, axis = 1)

        else : 
            tstdata=pd.read_csv(test_data, sep=separator_fm, header=None) # loading data without header
            X_index = []
            for i in range(len(tstdata.columns)): 
                if i == res_pos-1 : continue
                X_index.append(i)
            y_tst = tstdata.iloc[:,res_pos-1]
            X_tst = tstdata.iloc[:,X_index]
        
        return y_tst, X_tst
    def __variance_for_numeric(self,X, y, crt):
        crt_bigger = X >= crt
        crt_smaller = X < crt

        v1 = np.var(y[crt_bigger])
        v2 = np.var(y[crt_smaller])

        w1 = len(y[crt_bigger]) / len(y)
        w2 = len(y[crt_smaller]) / len(y)

        w_s_variance = w1 * v1 + w2 * v2
        return w_s_variance
    
    def __best_variance_for_numeric(self,x_cl,y):
        x_cl_sort = np.sort(x_cl.unique())

        # make criteria of this X variable
        classification_criteria = []
        for i in range(len(x_cl_sort)-1):
            classification_criteria.append(np.median([x_cl_sort[i],x_cl_sort[i+1]]))

        # find best variance criteria of this X variable
        variance_hist = []
        for crt in classification_criteria:
            variance_hist.append(self.__variance_for_numeric(x_cl,y,crt))

        crit = classification_criteria[np.argmin(variance_hist)]
        variance = variance_hist[np.argmin(variance_hist)]

        return crit, variance
    
    def __all_subset(self, xtype):
        listsub = list(xtype)
        subsets = []
        for i in range(2**len(listsub)):
            subset = []
            for k in range(len(listsub)):            
                if i & 1<<k:
                    subset.append(listsub[k])
            subsets.append(subset)        
        return subsets[1:len(subsets)-1]
    
    def __find_subset_index(self,tsubset, t):
        subset_index = []
        for i in range(len(tsubset)):
            temp = []
            for j in tsubset[i]:
                [temp.append(k) for k in list(t[t==j].index)]
                temp.sort()
            subset_index.append(temp)
        return subset_index
    
    def __find_subset_index2(self,tsubset, t):
        subset_index = []
        for j in tsubset:
            [subset_index.append(k) for k in list(t[t==j].index)]
            subset_index.sort()
        return subset_index
    
    def __variance_nominal(self,y,idx):
        notidx = [x for x in range(len(y)) if not(x in idx)]

        v1 = np.var(y[idx])
        v2 = np.var(y[notidx])

        w1 = len(y[idx]) / len(y)
        w2 = len(y[notidx]) / len(y)

        w_s_variance = w1 * v1 + w2 * v2
        return w_s_variance
    
    def __best_variance_for_nominal(self,x_cl,y):
        xtype = list(np.sort(x_cl.unique()))
        x_cl_subsets = self.__all_subset(xtype)
        x_cl_subsets_index = self.__find_subset_index(x_cl_subsets,x_cl)

        variance_hist = []
        for i in range(len(x_cl_subsets)):
            variance_hist.append(self.__variance_nominal(y,x_cl_subsets_index[i]))
        crit = x_cl_subsets[np.argmin(variance_hist)]
        variance = variance_hist[np.argmin(variance_hist)]

        return crit,variance
    
    def __CART_fit(self,X,y):
        crit = []
        variance = []
        catg = list(X.dtypes)
        for i in range(len(X.columns)):
            x_cl = X.iloc[:,i]
            if not(catg[i] == 'O'):
                temp = self.__best_variance_for_numeric(x_cl,y)
                crit.append(temp[0]) ; variance.append(temp[1])
            else:
                temp = self.__best_variance_for_nominal(x_cl,y)
                crit.append(temp[0]) ; variance.append(temp[1])

        #### self로 저장할 것들 ####
        bestvariable= np.argmin(variance)
        bestcrit = crit[bestvariable]
        bestvariance = variance[bestvariable]
        bestX = X.iloc[:,bestvariable]
        is_numeric = not(catg[bestvariable] == 'O')


        y_classified = []
        if is_numeric:
            y_classified.append(y[bestX >= bestcrit])
            y_classified.append(y[bestX < bestcrit])
        else:
            idx = find_subset_index2(bestcrit,bestX)
            notidx = [x for x in range(len(y)) if not(x in idx)]
            y_classified.append(y[idx])
            y_classified.append(y[notidx])

        y_pred = []
        y_pred.append(np.mean(y_classified[0]))
        y_pred.append(np.mean(y_classified[1]))

        self.bestvariable = bestvariable
        self.bestcrit = bestcrit
        self.bestvariance = bestvariance
        self.is_numeric = is_numeric
        self.y_classified = y_classified
        self.y_pred = y_pred
        pass

In [3]:
mod = CART_reg()

In [4]:
mod.fit()

Enter the name of training data file [(ex) boston_tr.csv] :  boston_tr.csv
Select the data Seperator(1 = ' ' or 2 = ','):  2
Does the data have column header? (y/n) :  y
Enter the column name of response variable among ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'tax', 'ptratio', 'black', 'lstat', 'medv'] :  medv


In [5]:
mod.predict()

Enter the name of test data file [(ex) boston_tst.csv] :  boston_tst.csv


,Actual Class,Predicted Class
0,21.6,27.656667
1,34.7,27.656667
2,27.1,17.147668
3,18.9,17.147668
4,18.9,17.147668
...,...,...
142,24.5,17.147668
143,19.7,17.147668
144,16.8,17.147668
145,23.9,27.656667


In [6]:
mod.print_results()

Enter the output file name to export [(ex) result.txt] :  result.txt
Enter the name of test data file [(ex) boston_tst.csv] :  boston_tst.csv


Tree Structure  

---------------------------------

	Node 1 : n = 343, mean = 21.743 

		Node 2 : lstat >= 10.14 , n = 193, mean = 17.148 

		Node 3 : lstat < 10.14 , n = 150, mean = 27.657 





Prediction Performance (Test)

---------------------------------

	MAE = 5.032 

	MAPE = 0.291 

	RMSE = 6.299 

